In [3]:
import json 
import requests
from typing import List, Dict
import time 
import re

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import accelerate
from transformers import pipeline

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )

Loading checkpoint shards: 100%|██████████| 8/8 [09:31<00:00, 71.44s/it]


In [ ]:
file_path = "/home/ltnga/LawVN-Instructction-Gen/src/data/data_test.json"
with open(file_path, 'r', encoding='utf-8') as f:
        data =json.load(f)

In [ ]:
articles = []
parts = text.split('Điều ')
for part in parts[1:]:
    article_text = 'Điều ' + part
    articles.append(article_text.strip())

In [ ]:
chunks = []
paragraphs = article.split('\n')
current_chunk = ""
    
for para in paragraphs:
    if len(current_chunk) + len(para) > max_tokens:
        if current_chunk:
            chunks.append(current_chunk)
        current_chunk = para
    else:
        current_chunk += "\n" + para if current_chunk else para
    
if current_chunk:
    chunks.append(current_chunk)

In [ ]:
 prompt = f"""Bạn là một chuyên gia về luật giao thông. Hãy đọc đoạn văn bản sau và tạo ra 5 cặp câu hỏi - câu trả lời về nội dung đoạn văn bản đó. Câu hỏi và câu trả lời phải chính xác, rõ ràng và có tính thực tế.

Văn bản:
{context}

Yêu cầu output format JSON:
{{
    "qas": [
        {{
            "question": "Câu hỏi...",
            "answer": "Câu trả lời...", 
            "context": "{context}",
            "metadata": {{
                "article": "Điều số...",
                "category": "Chọn 1 trong các category: Mức phạt/Hành vi vi phạm/Quy định chung",
                "tags": ["tag1", "tag2"]
            }}
        }}
    ]
}}

Chỉ trả về JSON, không thêm bất kỳ text nào khác."""

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2000,
        temperature=0.7,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
try:
    # Extract JSON content (everything between first { and last })
    json_str = response[response.find('{'):response.rfind('}')+1]
    json.loads(json_str)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Raw response: {response}")